# <font color="blue"> Cross-entropy on CartPole <font>

#### 사용하는 모델은 하나의 은닉층으로 구성 (입력층-은닉층-출력층)  
- 활성화 함수는 ReLU
- 은닉층의 뉴런 개수는 128 
- 하이퍼파라미터들은 대부분 tuning되지 않았으며 대부분 임의로 결정됨 

In [1]:
import gym 
from collections import namedtuple
import numpy as np
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16 
PERCENTILE = 70 

- HIDDEN_SIZE : 은닉층의 뉴런 개수 
- BATCH_SIZE : 반복마다 수행되는 에피소드의 개수 
- PERCENTILE : 에피소드의 총보상의 백분위수, 엘리트 에피소드를 거를 때 사용 (상위 30%)

In [3]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()#부모클래스의 생성자함수 호출
        
        self.net = nn.Sequential(nn.Linear(obs_size, hidden_size),
                                 nn.ReLU(),
                                 nn.Linear(hidden_size, n_actions))
        
    def forward(self, x):
        return self.net(x)

<img src ="./image/4/nn2.png" width=300>

#### 입력 : 4개 : 환경 관찰에 대한 벡터 
- 카트의 위치, 카트의 속도, 막대기의 각도, 막대기의 회전비


#### 출력 : 2개 : 수행할 수 있는 액션의 개수만큼 존재
- 왼쪽, 오른쪽 
- 우리가 원하는 네트워크의 출력은 액션의 확률분포 
- 마지막층에 softmax 비선형 함수 포함 필요 
    
    
    
    
####  하지만 구성한 네트워크에는 softmax가 포함되지 않음 
- Pytorch의 nn.CrossEntropyLoss 사용 
     - softmax와 cross-entropy를 합친 것 
     - 수치적으로 더 안정됨 
     - 네트워크로부터 생성된 비정규화된 값을 요구 (logit)
   

In [4]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

####  두 개의 helper class (collections 패키지의 named tuple) 
- EpisodeStep
  - 한 에피소드 내에서 에이전트가 수행하는 각각의 step
  - (observation, action) 
  - 엘리트 에피소드의 에피소드 스텝을 학습 데이터로 사용할 것 
    
    
    
- Episode 
  - 각 에피소드의 할인되지 않은 총보상과 EpisodeStep
  - (total reward, EpisodeStep)

In [5]:
Episode, EpisodeStep

(__main__.Episode, __main__.EpisodeStep)

### iterate_batches()
- 인자 
    - env : 환경 (Gym 라이브러리의 class instance) 
    - net : 네트워크
    - batch_size : 매 반복마다 생성되어야 할 에피소드의 개수 
    
    
    
- 데이터 
    - batch : (list) batch들을 모아놓은 리스트, Episode 객체를 모아놓은 리스트 
    - episode_reward : (float) 각 현재의 에피소드의 보상 
    - episode_step : (list) step들의 리스트, EpisodeStep 객체를 모아놓은 리스트 
    
    
    
- 첫번째 관찰을 얻기위해 환경 초기화 
- 네트워크의 출력을 액션의 확률분포로 변환하기위한 softmax 층 정의 

#### -  environment loop
- 매 반복마다, 현재의 관찰을 PyTorch 텐서로 변환한 뒤 네트워크로 전달하여 액션의 확률을 얻음 

#### 알아두어야 할 것 
- 관찰(observation)을 (1,4) 크기의 텐서로 변환 
    - PyTorch의 nn.Module 내에 있는 객체는 data item의 batch를 입력으로 받음 
    - CartPole 환경의 네개의 벡터를 하나의 리스트로 네트워크에 입력
    
    
- 네트워크의 출력에 softmax 함수를 적용해야 함 
     - 네트워크의 출력층에 비선형성을 사용하지 않았기 때문에 액션에 대한 raw score를 출력함 
     
     
- 텐서 unpacking 필요 
     - 네트워와 softmax 층은 가중치를 추적할 수 있는 텐서를 반환함 
     - .data 를 사용하여 텐서를 unpacking 한 후 numpy 배열로 변환
     - 이 배열은 입력과 같이 2차원 구조를 가짐 
         - 축 0은 batch의 차원
         - 그러므로, 확률분포의 1차원 벡터를 얻기 위해서는 batch의 첫번째 요소를 가져와야 함 ([0])
                        
####  현재의 에피소드가 끝나고 나면 (CartPole의 stick이 바닥에 떨어지면) 
- 최종 에피소드(Episode객체 (total reward, EpisodeStep))를  batch에 저장 

- episode_reward (보상누적기) 초기화

- episode_step 초기화 
- 환경을 초기화하여 다음 관찰로 사용 
    
####  batch가 바라는 에피소드 개수에 도달하면 
- yield가 수행됨 
  - yield : 함수 실행 중간에 빠져나올 수 있는 generator를 만들 때 사용 
    - 우리의 함수는 generator, 그러므로 yield 연산이 실행되면, 제어는 바깥 반복루프로 이동되며 yield 다음 라인이 수행됨 
  - batch를 초기화 함 
  
####  환경으로부터 얻은 관찰을 현재 관찰 변수로 정의 
- 같은 과정 반복

    - 관찰을 네트워크에 전달 
    - 수행할 액션을 선택 
    - 환경에 액션을 수행 
    - 이런 과정을 저장 

In [6]:
def iterate_batches(env, net, batch_size):
    """perparation"""
    batch = [] #list of 'Episode'
    episode_reward = 0.0 #reward counter for the current episode
    episode_steps = [] #list of "EpisodeStep"
    
    obs = env.reset()
    
    #softmax 객체###
    sm = nn.Softmax(dim=1)
    
    
    """environment loop"""
    while True :
        #observation
        obs_v = torch.FloatTensor([obs])
        
        #action probability
        #obs_v --> net --> softmax 
        #sm() __call__
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0] #(1, 4) --> (4, )
        
        #randomly choose action
        action = np.random.choice(len(act_probs), p=act_probs)
         
        #step
        next_obs, reward, is_done, _ = env.step(action)
        
        #env.render()
        
        #values append
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action)) #before obs, action
        
        """after the stick has fallen down"""
        if is_done : #is_done=True
            #appending which is successly done
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            
            #reset
            episode_reward = 0.0
            episode_steps = []
            
            next_obs = env.reset()
            
            
            if len(batch) == batch_size : #over than 16
                yield batch 
                
                #break the loop  after executing next line 
                batch = [] #clear batch 
        
        """after action ==> current obs"""
        obs = next_obs 

## training loop

#### - filter_batch()
- cross-entropy 방법의 중심부 
- 학습에 사용할 엘리트 에피소드를 걸러내는 함수 

#### 에피소드 필터링 
- 각 에피소드마다 에피소드가 보상경계보다 높은 총보상을 가지고 있는지 검사 
- 만약 어떤 에피소드의 총보상이 보상경계보다 더 높다면, 그 에피소드의 관찰과 액션을 학습시킬 리스트로 이동 
- example은 (reward, step) --> (total reward, EpisodeStep) --> (total_reward, (observation, action)) 
- map(lambda step : step.observation, example.steps)
    - example.steps의 observation 

In [9]:
def filter_batch(batch, percentile):
    """calculating reward"""
    rewards = list(map(lambda s :s.reward, batch))
    reward_bound = np.percentile(rewards, percentile) #boundary of elite 
    reward_mean = float(np.mean(rewards)) #just for monitoring 
    
    train_obs = []
    train_act = [] 
    
    """appending elite episodes"""
    for example in batch:##
        if example.reward < reward_bound : #에피소드 필터링 
            continue #pass #반복문의 처음으로 이동 
        #else 
        train_obs.extend(map(lambda step : step.observation, example.steps))
        train_act.extend(map(lambda step : step.action, example.steps))
        
    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    
    return train_obs_v, train_act_v, reward_bound, reward_mean

#### lambda(), map()
- lambda s : s.reward
    - lambda 인자 : 표현식 
- map(lambda s : s.reward, batch) 
    - map(함수, 리스트) 
    - 리스트의 원소를 하나씩 함수에 적용 
    
- map(lambda s : s.reward, batch) 
    - : batch의 원소들의 reward 값 

### Glue code
#### 마지막으로 모든 것을 하나로 합치고 대부분 training loop로 구성된 코드 
####  - batch 에피소드들의 보상평균 비교 
- 보상평균이 199보다 크가면 훈련을 멈춤 
- 왜 199?
     - Gym에서, CartPole환경은 마지막 100개 에피소드의 평균보상이 195보다 크다면 해결되었다 간주 
     - 하지만 우리의 방법은 너무 빨리 수렴하여 100 에피소드가 우리가 원하는 정도임 
     - CartPole환경은 200step으로 제한되어 있음 
          - 200step 이후에는 멈추도록 설정되어있음 
            

In [10]:
if __name__ == "__main__":
    env = gym.make("CartPole-v0")
    env = gym.wrappers.Monitor(env, directory="mon", force=True)
    
    obs_size = env.observation_space.shape[0]
    n_actions = env.action_space.n
    
    """creating net"""
    net = Net(obs_size, HIDDEN_SIZE, n_actions)
    
    """objective function"""
    objective = nn.CrossEntropyLoss()
    
    """optmizer"""
    optimizer = optim.Adam(params=net.parameters(), lr=0.01)
    
    """TensorboardX"""
    writer = SummaryWriter(comment='-cartpole')
    
    
    for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
        """filtering elite"""
        #obs from elite, act from elite, reward boundary, reward mean
        obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE) 
        
        optimizer.zero_grad()
        
        """send elite to net"""
        action_scores_v = net(obs_v)
        
        """loss"""
        loss_v = objective(action_scores_v, acts_v)
        loss_v.backward() 
        
        optimizer.step()
        
        print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" %(
               iter_no, loss_v.item(), reward_m, reward_b))
        
        
        
        writer.add_scalar("loss", loss_v.item(), iter_no)
        writer.add_scalar("reward_bound", reward_b, iter_no)
        writer.add_scalar("reward_mean", reward_m, iter_no)
        
        
        if reward_m > 199:
            print("Solved")
            break 
            
    writer.close()

0: loss=0.692, reward_mean=15.8, reward_bound=17.0
1: loss=0.671, reward_mean=31.2, reward_bound=41.0
2: loss=0.661, reward_mean=32.8, reward_bound=28.5
3: loss=0.647, reward_mean=40.0, reward_bound=45.5
4: loss=0.640, reward_mean=42.2, reward_bound=48.5
5: loss=0.641, reward_mean=44.8, reward_bound=41.0
6: loss=0.635, reward_mean=41.2, reward_bound=41.0
7: loss=0.622, reward_mean=34.9, reward_bound=41.5
8: loss=0.635, reward_mean=46.9, reward_bound=53.5
9: loss=0.611, reward_mean=44.2, reward_bound=46.5
10: loss=0.610, reward_mean=72.3, reward_bound=82.5
11: loss=0.588, reward_mean=60.4, reward_bound=63.0
12: loss=0.591, reward_mean=73.4, reward_bound=90.0
13: loss=0.593, reward_mean=69.4, reward_bound=78.0
14: loss=0.603, reward_mean=58.4, reward_bound=69.0
15: loss=0.585, reward_mean=59.4, reward_bound=69.5
16: loss=0.570, reward_mean=67.5, reward_bound=73.0
17: loss=0.562, reward_mean=83.6, reward_bound=85.5
18: loss=0.575, reward_mean=78.4, reward_bound=97.0
19: loss=0.575, reward


####  이것의 아이디어는 좋은 보상을 가져다주는 엘리트 액션을 수행하도록 네트워크를 강화하는 것 



#### 좋은 성능을 보임 

<img src="./image/4/t-1.png">


- 매 반복마다 16개의 에피소드를 수행하는 것 치고는 좋은 성능 
- TensorBoard를 통해 에이전트가 발전하는 것을 볼 수 있음
    - 보상경계 또한 매 batch마다 발전 (올라가고있음) 
    

####  Monitor
- 환경 생성의 다음 라인을 uncommenting하여 사용할 수 있음 
- (xvfb-run) 과 함께 재실행하면 프로그램은 각 학습 step마다의 비디오를 기록한 mon 디렉토리를 생성
- 주기적으로 에이전트의 행동을 구분된 비디오 파일로 기록 

    

## - 결론 

#### - 신경망은 관찰과 보상을 통해 단순히 환경에서 수행하는 것을 학습함 
- 관찰값에 대한 어떤한 해석도 없이 

#### - 다른 간단한 환경에도 적용 가능 
- 카트와 막대기로 이루어진 환경이 아니라 수량을 관찰하여 돈을 보상으로 얻는 창고 모델일 수도 있음 
    - 이 구현은 환경의 세부사항에 의존하지 않음 
    
    
#### - 다음 섹션에서는 Gym의 다른 환경에서 같은 방법이 어떻게 적용되는지 살펴볼 것 